# Needed Library Import
In this section the needed library to work with will be imported and some important variables will be initializatied

In [ ]:
# In order to make the analysis reproducible
RANDOM_SEED = 42

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=8f9d002f1aa094d3377bfe89bbc56f27c003ac0ca61688b57cc7bb67d6f8c2b2
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2Tn9uykXlYRjLcSLIgmv1iLdEvG_3gqA9M3V4hLGqH9N3JWEr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
import numpy as np
import math

# PySpark Initialization
In this section the connection with PySpark will be initializated

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=c730062f802637c706c0d342ce4029e755025d11ebc11049aed445c7cc636112
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RankingMetrics

In [ ]:
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("ALSAnalysis").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
port = '4050'
public_url = ngrok.connect(port).public_url

In [ ]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://5215-35-194-85-157.ngrok-free.app" -> "http://127.0.0.1:4050"


In [ ]:
spark

# Connection to Google Drive

In [ ]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Utils

The following function is used to compute the Recommender System evaluation metrics.

Each metric is computed for each eantry if the dataset (since the function will later on be applied to every row of the dataframe).

Each metric is implemented by only giving the implementation of the math formulation of the evaluation metric: specifically the function takes in input two values (that are lists in the dataset):


*   **predicted** which is the list of the values predicted for the entry of the database (meaning the items filtered, the one that were not the one used for the training)
*   **actual** which is the list of the values that corresponds to the ground truth for the entry of the dataset

The implementation of the formulas is the following:


*   **Precision@K** and **MAP**: the Precision@K (and consequently also the MAP) checks if the Recommender System is able to predict first the most important (most relevant) items for the user. The formula at the base of this two metrics is the same, the only difference resides in the fact that MAP is computed by computing Precision@K with differente cut-off points (meaning different Ks)

  $Precision@K$ = $\frac{relevant\_items\_in\_the\_first\_k\_positions}{k}$

  Where the relevant items are the one that the user interacted in a positive way with


*   **Recall@K**: the Recall@K checks if the Recommender System is able to provide sooner or later the relevant recommendations for the user (meaning that **regardless the order** is able to actually recommend the items with the user has interacted in a positive way. The formula of the metric is the following

  $Recall@K$ = $\frac{relevant\_items\_in\_the\_first\_k\_positions}{total\_relevant\_items}$


*   **NDCG**: the NDCG checks the relevance of recommended items and the position in which they were recommended. It values relevant recommendations that are closer to the top of the recommendation list, assigning higher scores to relevant items in higher positions. The formula for calculating the NDCG is the following

  $NDCG@K$ = $\frac{DCG@K}{IDCG@K}$

  Where DCG@K (Discounted Cumulative Gain) has the following formula
  
  $DCG@K$ = $∑_{i=1}^{k}\frac{2^{rel_i}-1}{log_2(i+1)}$

  And IDCG@K (Ideal Discounted Cumulative Gain) is the ideal DCG@K value where all the K items recommended are relevant



In [ ]:
def score(predicted, actual, metric, k=None):
    """
    This function is used to compute one of the 4 possible evaluation metrics
    thanks to which the model computed can be evaluated.
    Precisely the four metrics that can be computed are the following:
    - Precision: both MAP or Precision@K depending on the parameters given (if K
    is not specified MAP is computed)
    - Recall@K
    - NDCG
    """
    valid_metrics = ['precision', 'recall', 'ndcg']
    if metric not in valid_metrics:
        raise Exception(f"Choose one valid baseline in the list: {valid_metrics}")

    if metric == 'precision':
        if k is None:
          m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(k)
                     for k in range(1, len(predicted) + 1)])
        else:
          m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(k)])
    elif metric == 'recall':
        if k is None:
            raise ValueError("Parameter 'k' must be specified for recall calculation.")
        m = np.mean([float(len(set(predicted[:k]) & set(actual))) / float(len(actual))])
    elif metric == 'ndcg':
        v = [1 if i in actual else 0 for i in predicted]
        v_2 = [1 for i in actual]
        dcg = sum([(2**i-1)/math.log(k+2, 2) for (k, i) in enumerate(v)])
        idcg = sum([(2**i-1)/math.log(k+2, 2) for (k, i) in enumerate(v_2)])
        m = dcg/idcg

    return m


# ALS analysis
In the following section an analysis about the ALS model trained in *Videogame_Recommender_System.ipynb* will be performed.

Specifically the **RMSE** will be computed together with other useful metrics about the performance of the Recommender System like **MAP** (Mean Average Precision), **Average NDCG** (Average Normalized Discounted Cumulative Gain), **Precision@K** and **Recall@K**

In [ ]:
# Load the pre-traianed model
als_model = ALSModel.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/models/ALS19_08_2023__13_09_01')

In [ ]:
# Load the data about the users' activity
user_activity_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/user_activity',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [ ]:
# Inspect some row samples
user_activity_df.show(n=5)

+-----------------+-----------+---------------+------+------------+
|         steam_id|steam_appid|    recommended|rating|steam_int_id|
+-----------------+-----------+---------------+------+------------+
|76561198259233672|     951650|    Recommended|   3.0|           0|
|76561198341423566|    1581451|    Recommended|   3.0|           1|
|76561198031569409|       2310|    Recommended|   3.5|           2|
|76561198031569409|     337000|    Recommended|   5.0|           2|
|76561198031569409|    1353270|Not Recommended|   2.0|           2|
+-----------------+-----------+---------------+------+------------+
only showing top 5 rows



Just like in the model training it is necessary to divide the dataset in training and test set, in order to have the same RMSE computed during the training the same seed, stored in RANDOM_SEED, will be used

In [ ]:
(training, test) = user_activity_df.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

The tables with the predictions computed over the unkown data of the test set will be computed and in this way the RMSE will be presented

In [ ]:
predictions = als_model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

As we can see the model do not perform too poorley, since the RMSE is 0.94 and a good value to be expected is at most 0.85

In [ ]:
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 0.9436523803639781


There are two ways to compute recommendations with the model:

1.   **recommendForUserSubset**: with this function it is needed to provide a subset of users on which compute the recommendation
2.   **recommendForAllUsers**: with this function the recommendations are computed for all the users

It is important to notice that in both cases it is necessary to explicitly say how many recommendations must be computed



In [ ]:
# Create a subset with a single sample user (having as steam_int_id 168492)
user = spark.createDataFrame([168492], "int").toDF("steam_int_id")

In [ ]:
# Compute the recommendation for the user
steam_id = 168492
user_recs = als_model.recommendForUserSubset(user, 5)
print(f"Top 5 recommendations for User {steam_id}:")
user_recs.show(truncate=False)

Top 5 recommendations for User 168492:
+------------+----------------------------------------------------------------------------------------------------------+
|steam_int_id|recommendations                                                                                           |
+------------+----------------------------------------------------------------------------------------------------------+
|168492      |[{1145020, 5.032404}, {1487360, 5.020478}, {2134970, 5.0194483}, {2301630, 5.016604}, {1096060, 5.002679}]|
+------------+----------------------------------------------------------------------------------------------------------+



In [ ]:
# Generate top-k recommendations for each user
userRecs = als_model.recommendForAllUsers(5)  # Top-5 recommendations for each user

# Prepare the input for RankingMetrics
user_ground_truth = test.groupby('steam_int_id').agg(sqlf.collect_list('steam_appid').alias('ground_truth_items'))
user_train_items = training.groupby('steam_int_id').agg(sqlf.collect_list('steam_appid').alias('train_items'))

# Join the recommendations and ground truth data on the user ID
user_eval = userRecs.join(user_ground_truth, on='steam_int_id').join(user_train_items, on='steam_int_id') \
    .select('steam_int_id', 'recommendations.steam_appid', 'ground_truth_items', 'train_items', 'recommendations.rating')
user_eval = user_eval.toPandas()

In [ ]:
# Create the lists of the items and the ratings (that corresponds in the indexes
# meaning that in position i of the list there is an item j for which corresponds
# a rating k that is in position i in the list of the ratings) that were not used
# for the training of the model
user_eval['itemIndex_filtered'] = user_eval.apply(lambda x:[b for (b,z) in zip(x.steam_appid, x.rating) if b not in x.train_items], axis=1)
user_eval['rating_filtered'] = user_eval.apply(lambda x:[z for (b,z) in zip(x.steam_appid, x.rating) if b not in x.train_items], axis=1)

In [ ]:
# Compute MAP, NDCG, Recall@K and Precision@K with the previous function called
# 'score'
user_eval['precision'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'precision'), axis=1)
user_eval['NDCG'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'ndcg'), axis=1)

In [ ]:
user_eval['recall'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'recall', 5), axis=1)
user_eval['precisionAt'] = user_eval.apply(lambda x: score(x.itemIndex_filtered, x.ground_truth_items, 'precision', 5), axis=1)

In [ ]:
MAP = user_eval.precision.mean()
avg_NDCG = user_eval.NDCG.mean()
recall = user_eval.recall.mean()
precision = user_eval.precisionAt.mean()

As we can see from the values computed the model, even though it has a not-so-bad RMSE, it has some problems during the recommendation computation since the values for the metrics are the following:


*   MAP: 0.0000029
*   avg_NDCG: 0.0000022
*   Recall@K (K=5): 0.0000023
*   Precision@K (K=5): 0.0000033





In [ ]:
print(f'The Mean Average Precision (MAP) is: {MAP}')
print(f'The Average Normalized Discounted Cumulative Gain (avg_NDCG) is {avg_NDCG}')
print(f'The Recall@K with K=5 is {recall}')
print(f'The Precision@K with K=5 is {precision}')

The Mean Average Precision (MAP) is: 2.9196108798761012e-05
The Average Normalized Discounted Cumulative Gain (avg_NDCG) is 2.203468619847804e-05
The Recall@K with K=5 is 2.3468331061438896e-05
The Precision@K with K=5 is 3.2910657245184406e-05


# Create the data for the Demo
In this section the data sample for the demo is created; in order to make the web application as light as possible in the loadings it has been chosen to reduce the sample of players for which compute the recommendation from (more or less) 131000 to 20

In [ ]:
top_20_distinct_values = user_activity_df.groupBy('steam_id').count().orderBy(sqlf.col("count").desc()).limit(20)

In [ ]:
demo_df = user_activity_df.join(top_20_distinct_values, on='steam_id', how="inner")

In [ ]:
demo_df = demo_df.drop("count")

In [ ]:
demo_df.write.options(header='True', delimiter=',').csv('/content/gdrive/MyDrive/VideogameRecommenderSystem/demo_data')